In [29]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import pandas as pd ## pandas is used to manupulate the dataframe
import numpy as np ## numpy is used to do scientific calculations
import matplotlib.pyplot as plt ## matplotlib used for visualization
import seaborn as sns ## seaborn used for visualization 
import missingno as msno ## used to visualize missing values
import tensorflow as tf
import warnings ## used to remove warnings
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.model_selection import train_test_split,cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression,Lasso,Ridge,BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

import math
from IPython.display import Image
import warnings
warnings.filterwarnings("ignore")
sns.set(rc={"figure.figsize": (20, 15)})
sns.set_style("whitegrid")

Let's check my device

In [30]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8271380108029505063
 xla_global_id: -1]

In [31]:
df = pd.read_csv('input/clean_df.csv')

In [32]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,...,112,0,0,0,0,4,2010,WD,Normal,142125


In [33]:
train = df
print('Shape of the train data with all features:', train.shape)
train = train.select_dtypes(exclude=['object'])
print("")
print('Shape of the train data with numerical features:', train.shape)
train.drop('Id',axis = 1, inplace = True)

test = pd.read_csv('input/test.csv')
test = test.select_dtypes(exclude=['object'])

test.fillna(0,inplace=True)
test.drop('Id',axis = 1, inplace = True)
print('')
print('Shape of the test data:', test.shape)

Shape of the train data with all features: (1460, 77)

Shape of the train data with numerical features: (1460, 38)

Shape of the test data: (1459, 36)


## Data Preprocessing done in previous notebook, let's concentrate of turning our data into a matrix


In [34]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [35]:
X = train.drop(['SalePrice'], axis = 1)
Y = train['SalePrice']


In [43]:
enc = OneHotEncoder(handle_unknown = 'ignore')
enc.fit_transform(X)
enc.fit_transform(test)

<1459x7516 sparse matrix of type '<class 'numpy.float64'>'
	with 52524 stored elements in Compressed Sparse Row format>

In [44]:
scaler = StandardScaler()
scaler.fit_transform(X)
scaler.fit_transform(test)

array([[-0.87471081,  0.68484859,  0.36392912, ..., -0.09224354,
        -0.03828073,  1.71390537],
       [-0.87471081,  0.71585154,  0.89786065, ..., 19.73043821,
        -0.03828073,  1.71390537],
       [ 0.06135085,  0.49883089,  0.80964587, ..., -0.09224354,
        -1.1406145 ,  1.71390537],
       ...,
       [-0.87471081,  3.16508463,  2.05514965, ..., -0.09224354,
         1.06405305, -1.35995811],
       [ 0.64638939,  0.12679548,  0.12552719, ...,  1.01782664,
         0.32916387, -1.35995811],
       [ 0.06135085,  0.49883089, -0.03879049, ..., -0.09224354,
         1.79894223, -1.35995811]])

In [47]:
x_train,x_test, y_train, y_test =  train_test_split(X,Y, test_size = 0.1)

## Let's build a model!

In [65]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(64, activation = 'relu', input_shape = (X.shape[1],)))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(32, activation = 'relu'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer = 'adam', loss = 'mean_absolute_error')
model.fit(x_train, y_train, epochs= 5000, batch_size = 32)
test_loss = model.evaluate(x_test,y_test)


Epoch 1/5000
42/42 [==============================] - 1s 2ms/step - loss: 175346.3438
Epoch 2/5000
42/42 [==============================] - 0s 2ms/step - loss: 153157.4375
Epoch 3/5000
42/42 [==============================] - 0s 2ms/step - loss: 93806.0547
Epoch 4/5000
42/42 [==============================] - 0s 2ms/step - loss: 51141.1992
Epoch 5/5000
42/42 [==============================] - 0s 2ms/step - loss: 49421.4258
Epoch 6/5000
42/42 [==============================] - 0s 2ms/step - loss: 48283.4883
Epoch 7/5000
42/42 [==============================] - 0s 2ms/step - loss: 46574.8789
Epoch 8/5000
42/42 [==============================] - 0s 2ms/step - loss: 45138.4570
Epoch 9/5000
42/42 [==============================] - 0s 2ms/step - loss: 43844.7656
Epoch 10/5000
42/42 [==============================] - 0s 2ms/step - loss: 42200.5430
Epoch 11/5000
42/42 [==============================] - 0s 2ms/step - loss: 41001.6758
Epoch 12/5000
42/42 [==============================] - 0s 2ms

42/42 [==============================] - 0s 2ms/step - loss: 25955.0039
Epoch 97/5000
42/42 [==============================] - 0s 2ms/step - loss: 26220.6426
Epoch 98/5000
42/42 [==============================] - 0s 2ms/step - loss: 25958.4082
Epoch 99/5000
42/42 [==============================] - 0s 2ms/step - loss: 25989.8164
Epoch 100/5000
42/42 [==============================] - 0s 2ms/step - loss: 25920.8301
Epoch 101/5000
42/42 [==============================] - 0s 2ms/step - loss: 25830.7461
Epoch 102/5000
42/42 [==============================] - 0s 2ms/step - loss: 25744.0938
Epoch 103/5000
42/42 [==============================] - 0s 2ms/step - loss: 25775.6035
Epoch 104/5000
42/42 [==============================] - 0s 2ms/step - loss: 25914.0176
Epoch 105/5000
42/42 [==============================] - 0s 2ms/step - loss: 26074.2559
Epoch 106/5000
42/42 [==============================] - 0s 2ms/step - loss: 25817.3184
Epoch 107/5000
42/42 [==============================] - 0s 2m

42/42 [==============================] - 0s 2ms/step - loss: 24728.5215
Epoch 191/5000
42/42 [==============================] - 0s 2ms/step - loss: 24777.1562
Epoch 192/5000
42/42 [==============================] - 0s 2ms/step - loss: 24712.6992
Epoch 193/5000
42/42 [==============================] - 0s 2ms/step - loss: 24775.9434
Epoch 194/5000
42/42 [==============================] - 0s 2ms/step - loss: 24722.7617
Epoch 195/5000
42/42 [==============================] - 0s 2ms/step - loss: 24760.3613
Epoch 196/5000
42/42 [==============================] - 0s 2ms/step - loss: 24665.7148
Epoch 197/5000
42/42 [==============================] - 0s 2ms/step - loss: 24632.8770
Epoch 198/5000
42/42 [==============================] - 0s 2ms/step - loss: 24700.3164
Epoch 199/5000
42/42 [==============================] - 0s 2ms/step - loss: 24710.3691
Epoch 200/5000
42/42 [==============================] - 0s 2ms/step - loss: 24696.6172
Epoch 201/5000
42/42 [==============================] - 0s

42/42 [==============================] - 0s 2ms/step - loss: 24053.3945
Epoch 285/5000
42/42 [==============================] - 0s 2ms/step - loss: 24007.8711
Epoch 286/5000
42/42 [==============================] - 0s 2ms/step - loss: 24014.0156
Epoch 287/5000
42/42 [==============================] - 0s 2ms/step - loss: 24063.3672
Epoch 288/5000
42/42 [==============================] - 0s 2ms/step - loss: 24035.9180
Epoch 289/5000
42/42 [==============================] - 0s 2ms/step - loss: 24164.7422
Epoch 290/5000
42/42 [==============================] - 0s 2ms/step - loss: 24097.6543
Epoch 291/5000
42/42 [==============================] - 0s 2ms/step - loss: 24257.2578
Epoch 292/5000
42/42 [==============================] - 0s 2ms/step - loss: 24051.2754
Epoch 293/5000
42/42 [==============================] - 0s 2ms/step - loss: 24126.5586
Epoch 294/5000
42/42 [==============================] - 0s 2ms/step - loss: 24034.8652
Epoch 295/5000
42/42 [==============================] - 0s

42/42 [==============================] - 0s 2ms/step - loss: 23649.2715
Epoch 379/5000
42/42 [==============================] - 0s 2ms/step - loss: 23504.0098
Epoch 380/5000
42/42 [==============================] - 0s 2ms/step - loss: 23522.5957
Epoch 381/5000
42/42 [==============================] - 0s 2ms/step - loss: 23517.7676
Epoch 382/5000
42/42 [==============================] - 0s 2ms/step - loss: 23675.7207
Epoch 383/5000
42/42 [==============================] - 0s 2ms/step - loss: 24129.9414
Epoch 384/5000
42/42 [==============================] - 0s 2ms/step - loss: 23615.7676
Epoch 385/5000
42/42 [==============================] - 0s 2ms/step - loss: 23575.6738
Epoch 386/5000
42/42 [==============================] - 0s 2ms/step - loss: 23554.0312
Epoch 387/5000
42/42 [==============================] - 0s 2ms/step - loss: 24163.1699
Epoch 388/5000
42/42 [==============================] - 0s 2ms/step - loss: 23533.0664
Epoch 389/5000
42/42 [==============================] - 0s

...

42/42 [==============================] - 0s 2ms/step - loss: 12042.4189
Epoch 4570/5000
42/42 [==============================] - 0s 2ms/step - loss: 11393.7607
Epoch 4571/5000
42/42 [==============================] - 0s 2ms/step - loss: 12279.3223
Epoch 4572/5000
42/42 [==============================] - 0s 2ms/step - loss: 11616.8389
Epoch 4573/5000
42/42 [==============================] - 0s 2ms/step - loss: 11263.6533
Epoch 4574/5000
42/42 [==============================] - 0s 2ms/step - loss: 11621.7910
Epoch 4575/5000
42/42 [==============================] - 0s 3ms/step - loss: 12676.6182
Epoch 4576/5000
42/42 [==============================] - 0s 2ms/step - loss: 11767.0117
Epoch 4577/5000
42/42 [==============================] - 0s 2ms/step - loss: 11221.7910
Epoch 4578/5000
42/42 [==============================] - 0s 2ms/step - loss: 11376.6641
Epoch 4579/5000
42/42 [==============================] - 0s 2ms/step - loss: 11664.7783
Epoch 4580/5000
42/42 [=========================

42/42 [==============================] - 0s 2ms/step - loss: 11492.0586
Epoch 4663/5000
42/42 [==============================] - 0s 2ms/step - loss: 11974.6484
Epoch 4664/5000
42/42 [==============================] - 0s 2ms/step - loss: 11503.1377
Epoch 4665/5000
42/42 [==============================] - 0s 2ms/step - loss: 12195.6777
Epoch 4666/5000
42/42 [==============================] - 0s 2ms/step - loss: 11393.3496
Epoch 4667/5000
42/42 [==============================] - 0s 2ms/step - loss: 11047.0537
Epoch 4668/5000
42/42 [==============================] - 0s 2ms/step - loss: 11422.5889
Epoch 4669/5000
42/42 [==============================] - 0s 2ms/step - loss: 11682.9844
Epoch 4670/5000
42/42 [==============================] - 0s 2ms/step - loss: 11491.5293
Epoch 4671/5000
42/42 [==============================] - 0s 2ms/step - loss: 12271.4189
Epoch 4672/5000
42/42 [==============================] - 0s 2ms/step - loss: 11856.5225
Epoch 4673/5000
42/42 [=========================

42/42 [==============================] - 0s 2ms/step - loss: 11792.6191
Epoch 4756/5000
42/42 [==============================] - 0s 2ms/step - loss: 11553.5586
Epoch 4757/5000
42/42 [==============================] - 0s 2ms/step - loss: 10980.9277
Epoch 4758/5000
42/42 [==============================] - 0s 2ms/step - loss: 11890.3877
Epoch 4759/5000
42/42 [==============================] - 0s 2ms/step - loss: 12407.9824
Epoch 4760/5000
42/42 [==============================] - 0s 2ms/step - loss: 12612.9141
Epoch 4761/5000
42/42 [==============================] - 0s 2ms/step - loss: 11399.0459
Epoch 4762/5000
42/42 [==============================] - 0s 2ms/step - loss: 11359.4766
Epoch 4763/5000
42/42 [==============================] - 0s 2ms/step - loss: 11531.6836
Epoch 4764/5000
42/42 [==============================] - 0s 2ms/step - loss: 11406.5615
Epoch 4765/5000
42/42 [==============================] - 0s 2ms/step - loss: 12103.3291
Epoch 4766/5000
42/42 [=========================

42/42 [==============================] - 0s 2ms/step - loss: 11232.4541
Epoch 4849/5000
42/42 [==============================] - 0s 2ms/step - loss: 11356.0850
Epoch 4850/5000
42/42 [==============================] - 0s 2ms/step - loss: 11043.9814
Epoch 4851/5000
42/42 [==============================] - 0s 2ms/step - loss: 11568.1719
Epoch 4852/5000
42/42 [==============================] - 0s 2ms/step - loss: 11263.7344
Epoch 4853/5000
42/42 [==============================] - 0s 2ms/step - loss: 11464.4961
Epoch 4854/5000
42/42 [==============================] - 0s 2ms/step - loss: 12591.2568
Epoch 4855/5000
42/42 [==============================] - 0s 2ms/step - loss: 12588.3809
Epoch 4856/5000
42/42 [==============================] - 0s 2ms/step - loss: 11231.3057
Epoch 4857/5000
42/42 [==============================] - 0s 2ms/step - loss: 11324.4746
Epoch 4858/5000
42/42 [==============================] - 0s 2ms/step - loss: 10889.8281
Epoch 4859/5000
42/42 [=========================

42/42 [==============================] - 0s 2ms/step - loss: 11022.4512
Epoch 4942/5000
42/42 [==============================] - 0s 2ms/step - loss: 10714.0254
Epoch 4943/5000
42/42 [==============================] - 0s 2ms/step - loss: 10946.7979
Epoch 4944/5000
42/42 [==============================] - 0s 2ms/step - loss: 10809.8203
Epoch 4945/5000
42/42 [==============================] - 0s 2ms/step - loss: 10862.5488
Epoch 4946/5000
42/42 [==============================] - 0s 2ms/step - loss: 11037.7656
Epoch 4947/5000
42/42 [==============================] - 0s 2ms/step - loss: 11207.3262
Epoch 4948/5000
42/42 [==============================] - 0s 2ms/step - loss: 11276.7188
Epoch 4949/5000
42/42 [==============================] - 0s 2ms/step - loss: 10839.5186
Epoch 4950/5000
42/42 [==============================] - 0s 2ms/step - loss: 11502.9746
Epoch 4951/5000
42/42 [==============================] - 0s 2ms/step - loss: 10972.2949
Epoch 4952/5000
42/42 [=========================

In [66]:
from sklearn.metrics import r2_score
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)
r2_train= r2_score(y_train, y_train_pred)
r2_test= r2_score(y_test, y_test_pred)
print("R2 score for training set: ", r2_train)
print('')
print("R2 score for test set: ", r2_test)
train_loss = model.evaluate(x_train, y_train)
print("test_loss: ", test_loss)
test_loss = model.evaluate(x_test, y_test)
print("test_loss: ", test_loss)

5/5 [==============================] - 0s 2ms/step
R2 score for training set:  0.937605390924025

R2 score for test set:  0.9127137226065565
42/42 [==============================] - 0s 2ms/step - loss: 10257.5869
test_loss:  16752.935546875
5/5 [==============================] - 0s 2ms/step - loss: 16752.9355
test_loss:  16752.935546875
